# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Изучение общей информации

Для начала импортируем все необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from collections import Counter

In [2]:
df = pd.read_csv('/datasets/data.csv')
df.info()
display(df.head(10))
display(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


<b>Выводы</b>

После использования метода `info()` мы обнаружили, что исходный датасет включает 21525 строк (случаи взятия кредитов). 
Отдельно обращаем внимание на столбцы:  
- "days_employed" - есть отрицательные (-18388.949901) или слишком большие значения (401755.400475 дней ~ 1000 лет)<br>
- "children" - есть отрицательные значения (-1). Также отметитим, что у некоторых заемщиков 20 детей, что представляется технической ошибкой в данных.

## Предобработка данных

### Обработка пропусков

Метод `info()` показал, что столбцы "days_employed" и "total_income" имеют пропуски или нулевые значения. Мы помним, что наша главная задача - выяснить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок - соответственно пропуски в указанных столбцах не оказывают прямого влияния на результаты нашего анализа. Тем не менее, представим количество пропусков в наглядном виде.

In [3]:
print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Попробуем выяснить причину их появления. Мы увидели, что кол-во пропусков в столбцах "days_employed" и "total_income"  одинаковое. Вероятно, пропуски присутствуют в одних и тех же строках. Проверим это через оператор "И".

In [4]:
print('Кол-во случаев с одновременными пропусками в столбцах "days_employed" и "total_income" составляет',
      len(df[df['days_employed'].isna() & df['total_income'].isna()]))

Кол-во случаев с одновременными пропусками в столбцах "days_employed" и "total_income" составляет 2174


Возвращенное значение равно 2174, что подтверждает наше предположение, что пропуски присуствуют в одних и тех же строках. Рассмотрим строки, в которых присутствуют пропущенные значения.

In [5]:
df[df['days_employed'].isna() | df['total_income'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Выборка не позволяет увидеть причину появления пропусков. Вероятно, заемщики просто не заполнили данную информацию по каким-то своим причинам. <br>
В заполненении пропусков ограничемся использованием медианных значений, так как информация в данных столбцах не влияет на решение поставленной задачи. <br>
Прежде чем сделать это, вспоминаем, что "days_employed" имеет отрицательные значения. Хоть задача этого и не требует, взгляним на эти значения.

In [6]:
display(df[df['days_employed'] <= 0].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи
10,2,-4171.483647,36,высшее,0,женат / замужем,0,M,компаньон,0,113943.491460,покупка недвижимости


Возможно, по каким-то причинам знак "минуса" в отрицательных значениях стоит случайно, и нужно взять модуль этих значений. Если бы эта информация была бы критически важной, мы бы улглубились в своем анализе, однако в данном случае просто заменим отрицательные значения по модулю.

In [7]:
df['days_employed'] = df['days_employed'].abs()
display(df['days_employed'].describe())

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

Наконец подставим медианные значения.

In [8]:
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
df['total_income'] = df['total_income'].fillna(df['total_income'].median())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


<b>Выводы</b>

Мы заполнили пропуски средними значениями и убедились, что их больше нет. 

### Замена типа данных

Столбцы "days_employed" и "total_income" имеют тип "float64". Мы можем без зазрения совести избавиться от цифр после запятой в столбце "days_employed", просто использовав `.astype(int)`. Столбец "total_income" мы сохраним в изначальном виде для сохранения точности. 

In [9]:
df['days_employed'] = df['days_employed'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


<b>Выводы</b>

Мы обнаружили два столюца где можно изменить тип данных, но сделали это только для стажа. 

### Обработка дубликатов

Для начала приведем все строковые значения к нижнему регистру

In [10]:
df = df.apply(lambda x: x.str.lower() if x.dtype == "object" else x)
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763.565419,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525.971920,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823.934197,образование
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425.938277,покупка жилья для семьи


Проверим кол-во дубликатов.

In [11]:
df.duplicated().sum()

71

In [12]:
df = df.drop_duplicates().reset_index()
df.duplicated().sum()

0

<b>Выводы</b>

Дубликаты скорее всего возникли по каким-то техническим причинам. Для того, чтобы найти "скрытые" дубликаты мы привели все значения к нижнему регистру. В итоге из датасета было удалено 71 дублирующая строка.

### Лемматизация

Для установления зависимости между целью и возвратом кредита в срок, нужно лемматизировать цели и привести их к ограниченному кол-во категорий.
Для начала рассмотрим, какие значения встречаются в столбце purpose, используя `value_counts()`

In [13]:
display(df['purpose'].value_counts())

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Чтобы определить категории также можно подсчитать частоту лемм. Для этого сделаем общий список лемм.

In [14]:
m = Mystem()
lemmas_list = []
for i in range(df.shape[0]):
    lemmas_list = lemmas_list + m.lemmatize(df.loc[i, 'purpose'])

print(Counter(lemmas_list))

Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


Как видно все значения так или иначе связаны с четыремя категориями:
- Оперции с автомобилем
- Операции с недвижимостью
- Свадьба
- Образование

Cоздадим функцию, которая на основе лемматизированных значений будет присваивать категории целям кредитам

In [15]:
def purpose_lemmatize(purpose):
    lemmas = m.lemmatize(purpose)
    if "автомобиль" in lemmas:
        return "операции с автомобилем"
    elif "жилье" in lemmas or 'недвижимость' in lemmas:
        return "операции с недвижимостью"
    elif "свадьба" in lemmas:
        return "свадьба"
    elif 'образование' in lemmas:
        return 'образование'
    else:
        return 'другое'

Применим функцию к столбцу с целями

In [16]:
df['purpose_lemmatize'] = df['purpose'].apply(purpose_lemmatize)

Проверим, все ли значения были учтены

In [17]:
display(df['purpose_lemmatize'].value_counts())

операции с недвижимостью    10811
операции с автомобилем       4306
образование                  4013
свадьба                      2324
Name: purpose_lemmatize, dtype: int64

<b>Выводы</b>

Кредиты в наибольшей степени связаны с операциями с недвижимостью, в наименьшей - свадьбами.

### Категоризация данных

Исходя из вопросов и задач, нам нужно категоризировать:
- кол-во детей
- семейное положение
- уровень дохода

#### Дети

Убирем данные, полученые, скорее всего, в результате опечаток

In [18]:
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)

Сделаем функцию для категоризации кол-ва детей

In [19]:
def children_status(children):
    if children == 0:
        return "Без детей"
    elif children < 3:
        return "1-2"
    elif children < 5:
        return "3-4"
    elif children == 5:
        return '5'

Применим эту функцию к новому столбцу "children_status"

In [20]:
df['children_status'] = df['children'].apply(children_status)
print(df['children_status'].unique())

['1-2' 'Без детей' '3-4' '5']


#### Доходы

Cделаем функцию для категоризации дохода в зависимости от квартиля

In [21]:
q1 = df['total_income'].quantile(q=0.25)
q2 = df['total_income'].quantile(q=0.5)
q3 = df['total_income'].quantile(q=0.75)
q4 = df['total_income'].quantile(q=1)


def income_status(total_income):
    if total_income < 0:
        return "Ошибка"
    elif total_income <= q1:
        return "Малообеспеченные"
    elif total_income <= q2:
        return "Ниже среднего"
    elif total_income <= q3:
        return "Выше среднего"
    elif total_income <= q4:
        return "Обеспеченные"


df['income_status'] = df['total_income'].apply(income_status)
print(df['income_status'].unique())

['Обеспеченные' 'Ниже среднего' 'Выше среднего' 'Малообеспеченные']


<b>Выводы</b>

Были прокатегоризироаны кол-во детей и доходы.

## Ответы на поставленные вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [22]:
children_status_pivot = np.round(df.pivot_table(index=['children_status'], values='debt', aggfunc='mean'), 3)
display(children_status_pivot)

,debt
children_status,
1-2,0.093
3-4,0.084
5,0.000
Без детей,0.075


Результаты сводной таблицы свидетельсвуют, что большее количество детей делает менее вероятным невозврат кредита. Однако дальнешее рассмотрение показывает, что это не совсем так:

In [23]:
children_status_pivot = df.pivot_table(index=['children'], values='debt', aggfunc='mean')
children_status_pivot = np.round(children_status_pivot.sort_values('debt', ascending=False), 2)  
display(children_status_pivot)

,debt
children,
4,0.10
2,0.09
1,0.09
3,0.08
0,0.08
5,0.00


<b>Выводы</b>

Данные противоречивы и скорее всего колеблятся в пределах погрешности: связи между количеством детей и вероятностью невозврата нет. Однако связь между отсутствием детей и возвратом кредита чуть более наглядна. Создается впечатление, что заемщик без детей  вернет деньги с чуть большей вероятностью, однако значимость данного фактора без связи с другими вряд ли велика, требуется дополнительное исследование.

In [24]:
family_status_pivot = df.pivot_table(index=['family_status'], columns=['children_status'], values='debt', aggfunc='mean', 
                                     fill_value=0, margins=True, margins_name='Вероятность')
family_status_pivot = family_status_pivot.iloc[:-1] 
family_status_pivot = np.round(family_status_pivot.sort_values('Вероятность', ascending=False),2)
display(family_status_pivot)

children_status,1-2,3-4,5,Без детей,Вероятность
family_status,,,,,
не женат / не замужем,0.12,0.20,0,0.09,0.10
гражданский брак,0.11,0.12,0,0.08,0.09
женат / замужем,0.09,0.07,0,0.07,0.08
в разводе,0.07,0.08,0,0.07,0.07
вдовец / вдова,0.10,0.00,0,0.06,0.07


<b>Выводы</b>

В целом данные свидетельствуют, что люди, не находящиеся и неходившиеся в официальном браке, чаще не платят в срок - банку стоит насторожиться. А если принимать во внимание дополнительно фактор детей, то можно сделать вывод, что ненаходившиеся в официальном браке заемщики с детьми могут создать самые большие проблемы для банка.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [25]:
income_status_pivot = df.pivot_table(index=['income_status'], values='debt', aggfunc='mean')
income_status_pivot = np.round(income_status_pivot.sort_values('debt', ascending=False), 2)
print(income_status_pivot)

                  debt
income_status         
Выше среднего     0.09
Ниже среднего     0.09
Малообеспеченные  0.08
Обеспеченные      0.07


<b>Выводы</b>

Удивительно, но заемщики с доходом выше среднего чаще всех имеют просрочки по выплатам. Вероятно, они берут в долг и большую сумму. Данные требуют дополнительного исследования.

### Как разные цели кредита влияют на его возврат в срок?

In [26]:
purpose_pivot = df.pivot_table(index=['purpose_lemmatize'], values='debt', aggfunc='mean')
purpose_pivot = np.round(purpose_pivot.sort_values('debt', ascending=False), 2)
print(purpose_pivot)

                          debt
purpose_lemmatize             
операции с автомобилем    0.09
образование               0.09
свадьба                   0.08
операции с недвижимостью  0.07


<b>Выводы</b>

Операции с недвижимостью внушает доверие.

## Общий вывод

С точки зрения банка, заемщик, находящиеся в браке, не имеющий детей и берущий кредит на операции с недвижимостью - хороший заемщик. А вот заемщики с детьми, но не имевшие брака, берущие кредит на автомобиль должны банк настораживать. Однако данные выводы требуют более детального исследования.